# Controlador de vuelo para vehículos aéreos no tripulados multi-rotor basado en técnicas de aprendizaje profundo

## Entrenamiento Red LSTM

### Javier Cárdenas - Uriel Carrero

## 1. Descripción del Dataset

#### Importar Librerías

In [3]:
import os
import sys
import random

import pandas as pd
pd.set_option('display.max_columns', None) #Para mostrar todas las columnas
import matplotlib.pyplot as plt

import numpy as np                  # Cómputo Numérico
print(np.__version__)
#assert (np.__version__=='1.19.5'), 'Versión incorrecta de numpy, por favor instale 1.19.5'
seed = 5
np.random.seed(seed)
np.seterr(divide = 'ignore')

1.19.5


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
os.environ['TF_KERAS'] = '1'
import keras as kr
import keras_tuner as kt

import tensorflow as tf
from tensorflow.keras import models, layers
print(tf.__version__)
#assert (tf.__version__=='2.5.0'), 'Versión incorrecta de Tensorflow, por favor instale 2.5.0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from IPython.display import clear_output

2.5.0
Num GPUs Available:  1


In [5]:
import keras_tuner as kt

In [6]:
print(kt.__version__)

1.0.3


In [3]:
gpus = tf.config.list_physical_devices('GPU')
config = ConfigProto()
if gpus:
    try:
        config.gpu_options.allow_growth = True
        tf.compat.v1.enable_eager_execution()

        os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
session = InteractiveSession(config=config)

1 Physical GPUs, 1 Logical GPUs


2022-02-09 13:21:31.896754: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-09 13:21:31.898915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 13:21:31.899279: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 13:21:31.899525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
import gc #garbage collector
import gc; gc.enable()

## Cargar Datos

### Leemos el Dataset

In [5]:
root = '../logs/Datasets/'
dataset_name = 'Dataset_Final'
rootdir = root+dataset_name
if not os.path.exists(rootdir):
    print(f"{rootdir} not exist")

In [6]:
df = pd.read_csv(os.path.join(rootdir, random.choice(os.listdir(rootdir))))
delete_list = ['timestamps',
               'Q1', 'Q2', 'Q3', 'Q4', 
               'uvx', 'uvy', 'uvz', 
               'up', 'uq', 
               'uwp', 'uwq', 'uwr']
df_list = df.columns.to_list()
rpm_list = [i for i in df_list if ("RPM" in i)]
states_list = [i for i in df_list if not ((i in delete_list) or (i in rpm_list))]
Ts = df['timestamps'][1]-df['timestamps'][0]
fs = 1/Ts

In [7]:
dataset = []
for filename in os.listdir(rootdir):
    if not filename.endswith(".csv"):
        continue
        
    df = pd.read_csv(os.path.join(rootdir, filename))
    df =  df.drop(delete_list, axis=1)
    x = df.drop(rpm_list, axis=1)
    y = df.drop(states_list, axis=1)
    dataset.append([x, y])

df = None
x = None
y= None

#### Normalización de Estados (Entradas) y Acciones (Salidas)

In [8]:
def Norm(df, df_desc):
    for prop in list(df.columns):
        try:
            # 1 ~ Mean  7 ~ Max  3 ~ Min
            df[prop] = (df[prop]-df_desc[prop]['mean'])/(df_desc[prop]['max']-df_desc[prop]['min'])
        except e:
            print(e)
    return df

In [9]:
norm_data_path = f"{root}/data_description_{dataset_name}.csv"
df_desc = pd.read_csv(norm_data_path, index_col=0)  
df_desc

,x,y,z,p,q,r,vx,vy,vz,wp,wq,wr,ax,ay,az,ap,aq,ar,RPM0,RPM1,RPM2,RPM3,ux,uy,uz,ur
count,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06,3.068233e+06
mean,6.352137e-03,3.631930e-03,8.701060e-01,8.113104e-04,7.632901e-04,1.239124e-02,-1.253432e-04,-2.641033e-04,1.198860e-03,4.372706e-06,1.434372e-04,2.558978e-03,-1.772714e-05,3.597617e-05,-2.469848e-05,1.224907e-04,5.466858e-05,-3.303912e-05,1.441162e+04,1.441290e+04,1.440977e+04,1.441093e+04,6.439717e-03,3.417250e-03,8.782731e-01,1.285404e-02
std,1.067096e-01,1.015022e-01,7.571069e-01,3.005110e-02,3.288610e-02,4.450773e-01,7.177616e-02,6.552093e-02,2.601583e-01,1.942947e-01,2.135545e-01,5.107082e-01,3.168708e-01,2.945918e-01,1.454656e+00,7.742095e+00,6.830287e+00,3.325449e+00,1.055152e+03,1.058528e+03,1.058532e+03,1.058966e+03,1.058331e-01,1.010920e-01,7.641660e-01,4.414466e-01
min,-8.166897e-01,-8.176113e-01,2.113373e-04,-4.872289e-01,-4.386099e-01,-3.141419e+00,-1.156056e+00,-8.621260e-01,-6.841123e+00,-1.045986e+01,-7.981405e+00,-7.181414e+00,-4.279933e+01,-3.656422e+01,-9.800000e+00,-2.510366e+03,-3.059815e+03,-2.531001e+02,9.440300e+03,9.440300e+03,9.440300e+03,9.440300e+03,-8.000000e-01,-8.000000e-01,0.000000e+00,-3.140685e+00
25%,-7.023093e-03,-4.566531e-03,3.597720e-01,-7.548420e-04,-1.295726e-03,-3.379343e-04,-2.080171e-03,-1.373874e-03,-1.070760e-02,-5.708022e-03,-8.702793e-03,-6.507613e-05,-1.356462e-02,-7.912034e-03,-4.078746e-02,-4.151471e-02,-6.752393e-02,-2.696169e-04,1.438274e+04,1.438057e+04,1.438241e+04,1.437958e+04,0.000000e+00,0.000000e+00,3.549988e-01,0.000000e+00
50%,9.264773e-05,8.164912e-06,9.371726e-01,0.000000e+00,2.258764e-17,0.000000e+00,-1.518968e-08,1.364815e-17,3.785519e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.865757e-05,0.000000e+00,0.000000e+00,0.000000e+00,1.446835e+04,1.446836e+04,1.446834e+04,1.446834e+04,0.000000e+00,0.000000e+00,9.364582e-01,0.000000e+00
75%,2.154385e-02,1.321506e-02,1.092425e+00,9.607826e-04,1.469981e-03,5.312049e-04,2.095210e-03,1.519517e-03,1.501362e-02,5.440229e-03,8.939743e-03,3.879907e-05,1.237756e-02,8.299066e-03,3.374101e-02,4.180967e-02,6.758204e-02,3.539114e-04,1.452948e+04,1.453062e+04,1.452831e+04,1.452817e+04,1.838854e-02,8.750144e-03,1.144531e+00,0.000000e+00
max,8.194435e-01,8.146467e-01,4.000000e+00,3.767827e-01,4.339304e-01,3.141577e+00,8.494385e-01,8.597021e-01,6.373837e+00,7.147506e+00,1.179763e+01,7.300762e+00,3.906528e+01,3.332884e+01,1.921161e+02,2.711637e+03,2.831431e+03,1.630572e+02,2.166645e+04,2.166645e+04,2.166645e+04,2.166645e+04,8.000000e-01,8.000000e-01,4.000000e+00,3.141519e+00


In [10]:
for i, data in enumerate(dataset):
    x, y = data
    Norm(x, df_desc)
    Norm(y, df_desc)
    dataset[i]=[x,y]

#### División del dataset para entrenamiento, validación, prueba

In [11]:
X = []
Y = []

for sample in dataset:
    x, y = sample
    X.append(x)
    Y.append(y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.06, random_state=42)
X_train,  X_val, Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=0.1)

del X
del Y

In [12]:
print(f'Total={len(dataset)}, Entrenamiento={len(X_train)} ({round(100*len(X_train)/len(dataset))}%),'\
      f'Validación={len(X_val)} ({round(100*len(X_val)/len(dataset))}%),'\
      f'Prueba={len(X_test)} ({round(100*len(X_test)/len(dataset))}%)')

Total=167, Entrenamiento=140 (84%),Validación=16 (10%),Prueba=11 (7%)


#### Generador de Ejemplos de entrenamiento

Entrenar un modelo con una señal de 50000 datos en cada iteración sería una tarea que tomaría demasiado tiempo, así mismo, cuando se necesite realizar la inferencia del modelo, se necesitaría esa misma cantidad de datos, por lo que no se utiliza toda la señal de entrenamiento, sino pequeños segmentos de tamaño N, por lo que se generarán M-N (longitud de toda la señal, 50000) señales de longitud N para el entrenamiento, lo que aumentaría el consumo de memoria. Por tal motivo se define un generador.

In [13]:
class DataGenerator:
    def __init__(self, X=[], Y=[], dataset = None, batch_size=512, window=512, sequence_out=False, variable_window=False, delta_window=1, feedback=False, window_feedback=1):
        if dataset:
            for data in dataset:
                X.append(data[0])
                Y.append(data[1])
            self.X = X
            self.Y = Y
        elif X and Y:
            if len(X)!=len(Y):
                raise Exception("La longitud de datos de X e Y deben ser iguales")
            self.X = X
            self.Y = Y
        else:
            raise Exception("Debe especificar dataset o X, Y")
            
        self.n  = len(X)                 ### Número de ejemplos de entrenamiento
        x_shape = X[0].shape
        y_shape = Y[0].shape
        self.batch_size = batch_size
        self.window = window
        self.variable_window = variable_window
        self.delta_window = delta_window
        self.feedback = feedback
        self.i  = x_shape[1] if not self.feedback else x_shape[1]+y_shape[1]   ### Número de características
        self.j  = y_shape[1]                                                   ### Número de salidas
        #self.window_feedback = window_feedback
        if self.variable_window:
            self.window_max = self.window+self.delta_window
            self.window_min = self.window-self.delta_window
            if self.window_min<1:
                raise IndexError(f'delta_window no puede ser igual o mayor a la ventana')
        self.sequence_out = sequence_out
        self.set_shapes()
        
    def set_shapes(self):
        if self.sequence_out:
            self.shapes = ((self.batch_size, self.window, self.i),
                           (self.batch_size, self.window, self.j))
        else:
            self.shapes = ((self.batch_size, self.window, self.i),
                           (self.batch_size, self.j))

    def buid_init(self):
        if self.variable_window:
            self.window = np.random.randint(self.window_min, self.window_max)
            self.set_shapes()
        self.samples = np.empty(shape= self.shapes[0], dtype='float32') 
        self.labels  = np.empty(shape= self.shapes[1], dtype='float32') 
        self.batchcount = 0
        
    def build_data(self):
        self.buid_init()
        if self.feedback:
            i_0 = 1
        else:
            i_0 = 0
        while True:
            try:
                index = np.random.randint(0, self.n-1)            ### Trayectoria a seleccionar
                m = len(self.X[index])          ### Número de steps por ejemplo
                if m-self.window-1<=0:
                    raise IndexError(f'El tamaño de la ventana es mayor a la trayectoria')
                else:
                    start_index = np.random.randint(i_0, int(m-self.window-1))
                    final_index = start_index+self.window
                    x = self.X[index][start_index:final_index].to_numpy()
                    if self.feedback:
                        y = self.Y[index][start_index-1:final_index-1].to_numpy()
                        self.samples[self.batchcount] = np.concatenate((x,y), axis=1)
                    else:
                        self.samples[self.batchcount] = x
                        
                    if self.sequence_out:
                        self.labels[self.batchcount] = self.Y[index][start_index:final_index].to_numpy()
                    else:
                        self.labels[self.batchcount] = self.Y[index].loc[final_index]
            except IndexError as e:
                print(f'ERROR: Ejemplo {self.batchcount}: {e}')
                raise e
                
            self.batchcount += 1
            if self.batchcount >= self.batch_size:
                yield self.samples.astype(np.float32), self.labels.astype(np.float32)
                self.buid_init()

In [14]:
window = 64                   ### Número de steps por ejemplo
batch_size = 512              ### Número de ejemplos por batch
sequence_out = False
variable_window=True
feedback = False
delta_window=window/3

In [15]:
train_generator = DataGenerator(X=X_train, Y=Y_train, batch_size=batch_size, window=window, sequence_out=sequence_out, variable_window=variable_window, delta_window=delta_window, feedback=feedback)
val_generator   = DataGenerator(X=X_val,   Y=Y_val,   batch_size=batch_size, window=window, sequence_out=sequence_out, variable_window=variable_window, delta_window=delta_window, feedback=feedback)
test_generator  = DataGenerator(X=X_test,  Y=Y_test,  batch_size=batch_size, window=window, sequence_out=sequence_out, variable_window=variable_window, delta_window=delta_window, feedback=feedback)

In [16]:
dataset_train = tf.data.Dataset.from_generator(train_generator.build_data,
                                        output_types = (tf.float32, tf.float32))
dataset_val = tf.data.Dataset.from_generator(val_generator.build_data,
                                        output_types = (tf.float32, tf.float32))
dataset_test = tf.data.Dataset.from_generator(test_generator.build_data,
                                        output_types = (tf.float32, tf.float32))

In [17]:
for _ in range(5):
    x, y = next(train_generator.build_data())
    print(f'x.shape={x.shape}, y.shape={y.shape}')

x.shape=(512, 64, 22), y.shape=(512, 4)
x.shape=(512, 83, 22), y.shape=(512, 4)
x.shape=(512, 56, 22), y.shape=(512, 4)
x.shape=(512, 77, 22), y.shape=(512, 4)
x.shape=(512, 64, 22), y.shape=(512, 4)


## Keras Model

## Callbacks

In [18]:
main_metric = 'mean_squared_error'
#metrics = [main_metric, 'cosine_similarity', 'logcosh']
metrics = main_metric

#### Definición del Modelo

In [19]:
input_dim = len(states_list) if not feedback else len(states_list)+len(rpm_list)
output_dim = len(rpm_list)
print(f'input_dim: {input_dim}, output_dim: {output_dim}')

input_dim: 22, output_dim: 4


In [20]:
class LSTMHyperModel(kt.HyperModel):
    def __init__(self, input_dim, output_dim, metrics='mean_squared_error', loss='mean_squared_error'):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.metrics = metrics
        self.loss = loss

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(layers.Conv1D(filters=hp.Int(f'Conv1_filters',min_value=32,max_value=512,step=32,default=128), 
                                        kernel_size=3, padding='same', 
                                        activation=hp.Choice(
                                        'Conv1_activation',
                                        values=['relu', 'tanh', 'sigmoid'],
                                        default='relu'),
                                input_shape=(None, self.input_dim)
                                       ),
                 )
        hidden_CNNlayers = hp.Int('Hidden_CNNlayers',min_value=1,max_value=5,step=1,default=5)
        for i in range(hidden_CNNlayers):
            model.add(layers.Conv1D(filters=hp.Int(f'Conv1_filters_{i}',min_value=32,max_value=512,step=32,default=128), 
                                            kernel_size=3, padding='same', 
                                            activation=hp.Choice(
                                            'Conv1_activation',
                                            values=['relu', 'tanh', 'sigmoid'],
                                            default='relu')
                                           ))
        
        hidden_LSTMlayers = hp.Int('Hidden_LSTMlayers',min_value=1,max_value=5,step=1,default=5)
        for i in range(hidden_LSTMlayers):
            if i==hidden_LSTMlayers-1:
                return_sequences = False
            else:
                return_sequences = True
                
            model.add(layers.LSTM(units=hp.Int(f'LSTM_Hidden_units_{i}',min_value=32,max_value=256,step=32,default=128),
                                  return_sequences=return_sequences))
        
        hidden_layers = hp.Int('Hidden_FClayers',min_value=1,max_value=5,step=1,default=5)
        for i in range(hidden_layers):
            model.add(layers.Dense(units=hp.Int(f'Hidden_units_{i}',min_value=32,max_value=512,step=32,default=128), 
                                   activation=hp.Choice(
                                        'dense_activation',
                                        values=['relu', 'tanh', 'sigmoid'],
                                        default='relu')))
        
        model.add(layers.Dense(self.output_dim))

        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss=self.loss,
            metrics=self.metrics
        )
        return model

In [21]:
hypermodel = LSTMHyperModel(input_dim, output_dim)

#### Tuner

In [22]:
SEED = 1 
MAX_TRIALS = 50
EXECUTION_PER_TRIAL = 2
HYPERBAND_MAX_EPOCHS = 150

tuner = kt.tuners.Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective=F'val_{main_metric}',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='./Models/hyperband/',
    project_name=f'{dataset_name}'
)

INFO:tensorflow:Reloading Oracle from existing project ./Models/hyperband/Dataset_Final/oracle.json


#### Compilado el Modelo

In [23]:
tuner.search_space_summary()

Search space summary
Default search space size: 24
LSTM_units (Int)
{'default': 256, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
Hidden_LSTMlayers (Int)
{'default': 5, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
Conv1D_layer (Boolean)
{'default': True, 'conditions': []}
Conv1_filters_0 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
Conv1_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
LSTM_Hidden_units_0 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
Conv1_filters_1 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
LSTM_Hidden_units_1 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
Conv1_filters_2 (Int)
{'default': 128,

#### Entrenamiento del Modelo

In [24]:
%%time
N_EPOCH_SEARCH = 150

#max_queue_size = maximum size of the internal training queue which is used to "precache" samples from the generator
#workers = number of threads generating batches in parallel. Batches are computed in parallel on the CPU and passed on the fly onto the GPU for neural network computations
tuner.search(dataset_train.repeat(),
            steps_per_epoch=len(X_train), # steps per epoch
            epochs=N_EPOCH_SEARCH,
            validation_data=dataset_val.repeat(),
            validation_steps=10,
            max_queue_size=2, ##
            workers=1,
            callbacks=[kr.callbacks.TensorBoard(f'../Models/tmp/tb_logs/{dataset_name}')]
            )

Trial 255 Complete [02h 23m 43s]
val_mean_squared_error: 0.0007204765861388296

Best val_mean_squared_error So Far: 0.00023271616373676807
Total elapsed time: 16h 04m 06s
INFO:tensorflow:Oracle triggered exit
CPU times: user 2d 14h 56min 21s, sys: 59min 10s, total: 2d 15h 55min 31s
Wall time: 2d 16h 4min 6s


#### Se guarda el Modelo

In [25]:
# Show a summary of the search
tuner.results_summary()

Results summary
Results in ./Models/hyperband/Dataset_Final
Showing 10 best trials
Objective(name='val_mean_squared_error', direction='min')
Trial summary
Hyperparameters:
LSTM_units: 256
Hidden_LSTMlayers: 5
Conv1D_layer: True
Conv1_filters_0: 224
Conv1_activation: tanh
LSTM_Hidden_units_0: 224
Conv1_filters_1: 416
LSTM_Hidden_units_1: 192
Conv1_filters_2: 288
LSTM_Hidden_units_2: 128
Conv1_filters_3: 384
LSTM_Hidden_units_3: 192
Conv1_filters_4: 288
LSTM_Hidden_units_4: 96
Hidden_FClayers: 3
Hidden_units_0: 448
dense_activation: tanh
Hidden_units_1: 192
Hidden_units_2: 224
Hidden_units_3: 224
Hidden_units_4: 512
learning_rate: 0.0010483453984810058
Conv1_filters: 224
Hidden_CNNlayers: 2
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 5cc6c63d02ce22152fd609c2caa7bba0
Score: 0.00023271616373676807
Trial summary
Hyperparameters:
LSTM_units: 480
Hidden_LSTMlayers: 2
Conv1D_layer: True
Conv1_filters_0: 128
Conv1_activation: relu
LSTM_Hidden_units_

In [ ]:
EPOCHS = 150
bestHPs = tuner.get_best_hyperparameters(5)
models = []
logs = []
I = 'Tuner'
for i, bestHP in enumerate(bestHPs):
    model = tuner.hypermodel.build(bestHP)
    history = model.fit(  dataset_train.repeat(), 
                epochs=EPOCHS, 
                steps_per_epoch = len(X_train),
                verbose=1,
                validation_data = dataset_val.repeat(),
                validation_steps= len(X_val)
             )
    model.save(f'../Models/{dataset_name}_{I}_{i}.h5',  include_optimizer=False)
    print(f'../Models/{dataset_name}_{I}_{i}.h5')
    models.append(model)
    logs.append(history)

Epoch 1/150
140/140 [==============================] - 40s 228ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0020 - val_mean_squared_error: 0.0020
Epoch 2/150
140/140 [==============================] - 31s 221ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0015 - val_mean_squared_error: 0.0015
Epoch 3/150
140/140 [==============================] - 31s 222ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 4/150
140/140 [==============================] - 31s 221ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 5/150
140/140 [==============================] - 31s 219ms/step - loss: 9.0098e-04 - mean_squared_error: 9.0098e-04 - val_loss: 0.0010 - val_mean_squared_error: 0.0010
Epoch 6/150
140/140 [==============================] - 31s 218ms/step - loss: 8.2985e-04 - mean_squared_error: 8.2985e-04 - val_loss: 0.0010 - val_mean_squared_er

Epoch 91/150
140/140 [==============================] - 31s 221ms/step - loss: 3.9120e-04 - mean_squared_error: 3.9120e-04 - val_loss: 4.3262e-04 - val_mean_squared_error: 4.3262e-04
Epoch 92/150
140/140 [==============================] - 31s 219ms/step - loss: 2.8703e-04 - mean_squared_error: 2.8703e-04 - val_loss: 5.4108e-04 - val_mean_squared_error: 5.4108e-04
Epoch 93/150
140/140 [==============================] - 31s 221ms/step - loss: 3.3623e-04 - mean_squared_error: 3.3623e-04 - val_loss: 4.7189e-04 - val_mean_squared_error: 4.7189e-04
Epoch 94/150
140/140 [==============================] - 30s 214ms/step - loss: 4.0442e-04 - mean_squared_error: 4.0442e-04 - val_loss: 5.4076e-04 - val_mean_squared_error: 5.4076e-04
Epoch 95/150
140/140 [==============================] - 31s 221ms/step - loss: 4.1374e-04 - mean_squared_error: 4.1374e-04 - val_loss: 5.8729e-04 - val_mean_squared_error: 5.8729e-04
Epoch 96/150
140/140 [==============================] - 31s 221ms/step - loss: 3.2238

140/140 [==============================] - 27s 194ms/step - loss: 5.6893e-04 - mean_squared_error: 5.6893e-04 - val_loss: 6.9084e-04 - val_mean_squared_error: 6.9084e-04
Epoch 31/150
140/140 [==============================] - 27s 194ms/step - loss: 5.8653e-04 - mean_squared_error: 5.8653e-04 - val_loss: 6.6190e-04 - val_mean_squared_error: 6.6190e-04
Epoch 32/150
140/140 [==============================] - 27s 191ms/step - loss: 6.5301e-04 - mean_squared_error: 6.5301e-04 - val_loss: 7.6855e-04 - val_mean_squared_error: 7.6855e-04
Epoch 33/150
140/140 [==============================] - 27s 192ms/step - loss: 4.5585e-04 - mean_squared_error: 4.5585e-04 - val_loss: 6.3874e-04 - val_mean_squared_error: 6.3874e-04
Epoch 34/150
140/140 [==============================] - 26s 188ms/step - loss: 4.7899e-04 - mean_squared_error: 4.7899e-04 - val_loss: 7.2161e-04 - val_mean_squared_error: 7.2161e-04
Epoch 35/150
140/140 [==============================] - 27s 192ms/step - loss: 5.3240e-04 - mean_s

Epoch 75/150
140/140 [==============================] - 27s 195ms/step - loss: 4.6671e-04 - mean_squared_error: 4.6671e-04 - val_loss: 6.6940e-04 - val_mean_squared_error: 6.6940e-04
Epoch 76/150
140/140 [==============================] - 27s 193ms/step - loss: 3.9260e-04 - mean_squared_error: 3.9260e-04 - val_loss: 3.9039e-04 - val_mean_squared_error: 3.9039e-04
Epoch 77/150
140/140 [==============================] - 28s 199ms/step - loss: 4.3896e-04 - mean_squared_error: 4.3896e-04 - val_loss: 5.1720e-04 - val_mean_squared_error: 5.1720e-04
Epoch 78/150
140/140 [==============================] - 27s 196ms/step - loss: 3.9084e-04 - mean_squared_error: 3.9084e-04 - val_loss: 6.8791e-04 - val_mean_squared_error: 6.8791e-04
Epoch 79/150
140/140 [==============================] - 28s 199ms/step - loss: 4.0180e-04 - mean_squared_error: 4.0180e-04 - val_loss: 6.0935e-04 - val_mean_squared_error: 6.0935e-04
Epoch 80/150
140/140 [==============================] - 27s 191ms/step - loss: 4.4461

Epoch 120/150
140/140 [==============================] - 28s 202ms/step - loss: 4.5562e-04 - mean_squared_error: 4.5562e-04 - val_loss: 3.1597e-04 - val_mean_squared_error: 3.1597e-04
Epoch 121/150
140/140 [==============================] - 28s 197ms/step - loss: 3.6338e-04 - mean_squared_error: 3.6338e-04 - val_loss: 5.5710e-04 - val_mean_squared_error: 5.5710e-04
Epoch 122/150
140/140 [==============================] - 28s 197ms/step - loss: 3.5507e-04 - mean_squared_error: 3.5507e-04 - val_loss: 3.9824e-04 - val_mean_squared_error: 3.9824e-04
Epoch 123/150
140/140 [==============================] - 27s 196ms/step - loss: 2.9417e-04 - mean_squared_error: 2.9417e-04 - val_loss: 4.4564e-04 - val_mean_squared_error: 4.4564e-04
Epoch 124/150
140/140 [==============================] - 27s 194ms/step - loss: 3.4472e-04 - mean_squared_error: 3.4472e-04 - val_loss: 5.8255e-04 - val_mean_squared_error: 5.8255e-04
Epoch 125/150
140/140 [==============================] - 28s 198ms/step - loss: 

...

Epoch 60/150
140/140 [==============================] - 26s 185ms/step - loss: 4.3505e-04 - mean_squared_error: 4.3505e-04 - val_loss: 4.3741e-04 - val_mean_squared_error: 4.3741e-04
Epoch 61/150
140/140 [==============================] - 26s 185ms/step - loss: 4.0717e-04 - mean_squared_error: 4.0717e-04 - val_loss: 3.8508e-04 - val_mean_squared_error: 3.8508e-04
Epoch 62/150
140/140 [==============================] - 26s 186ms/step - loss: 4.5471e-04 - mean_squared_error: 4.5471e-04 - val_loss: 5.6198e-04 - val_mean_squared_error: 5.6198e-04
Epoch 63/150
140/140 [==============================] - 26s 183ms/step - loss: 4.3334e-04 - mean_squared_error: 4.3334e-04 - val_loss: 3.4723e-04 - val_mean_squared_error: 3.4723e-04
Epoch 64/150
140/140 [==============================] - 26s 184ms/step - loss: 4.4136e-04 - mean_squared_error: 4.4136e-04 - val_loss: 5.5638e-04 - val_mean_squared_error: 5.5638e-04
Epoch 65/150
140/140 [==============================] - 26s 184ms/step - loss: 4.6792

Epoch 105/150
140/140 [==============================] - 26s 183ms/step - loss: 3.9888e-04 - mean_squared_error: 3.9888e-04 - val_loss: 6.6112e-04 - val_mean_squared_error: 6.6112e-04
Epoch 106/150
140/140 [==============================] - 25s 181ms/step - loss: 4.3289e-04 - mean_squared_error: 4.3289e-04 - val_loss: 4.5253e-04 - val_mean_squared_error: 4.5253e-04
Epoch 107/150
140/140 [==============================] - 25s 181ms/step - loss: 3.5671e-04 - mean_squared_error: 3.5671e-04 - val_loss: 5.2225e-04 - val_mean_squared_error: 5.2225e-04
Epoch 108/150
140/140 [==============================] - 25s 181ms/step - loss: 3.6809e-04 - mean_squared_error: 3.6809e-04 - val_loss: 4.3355e-04 - val_mean_squared_error: 4.3355e-04
Epoch 109/150
140/140 [==============================] - 26s 186ms/step - loss: 3.6845e-04 - mean_squared_error: 3.6845e-04 - val_loss: 4.5070e-04 - val_mean_squared_error: 4.5070e-04
Epoch 110/150
140/140 [==============================] - 25s 180ms/step - loss: 

140/140 [==============================] - 23s 166ms/step - loss: 5.2808e-04 - mean_squared_error: 5.2808e-04 - val_loss: 7.8335e-04 - val_mean_squared_error: 7.8335e-04
Epoch 45/150
140/140 [==============================] - 23s 165ms/step - loss: 4.6694e-04 - mean_squared_error: 4.6694e-04 - val_loss: 6.6292e-04 - val_mean_squared_error: 6.6292e-04
Epoch 46/150
140/140 [==============================] - 24s 170ms/step - loss: 5.3280e-04 - mean_squared_error: 5.3280e-04 - val_loss: 6.6944e-04 - val_mean_squared_error: 6.6944e-04
Epoch 47/150
140/140 [==============================] - 24s 171ms/step - loss: 5.0571e-04 - mean_squared_error: 5.0571e-04 - val_loss: 4.9533e-04 - val_mean_squared_error: 4.9533e-04
Epoch 48/150
140/140 [==============================] - 24s 175ms/step - loss: 5.2681e-04 - mean_squared_error: 5.2681e-04 - val_loss: 5.9985e-04 - val_mean_squared_error: 5.9985e-04
Epoch 49/150
140/140 [==============================] - 24s 170ms/step - loss: 5.2707e-04 - mean_s

Epoch 89/150
140/140 [==============================] - 24s 173ms/step - loss: 4.2699e-04 - mean_squared_error: 4.2699e-04 - val_loss: 7.3304e-04 - val_mean_squared_error: 7.3304e-04
Epoch 90/150
140/140 [==============================] - 24s 170ms/step - loss: 5.4576e-04 - mean_squared_error: 5.4576e-04 - val_loss: 5.7518e-04 - val_mean_squared_error: 5.7518e-04
Epoch 91/150
140/140 [==============================] - 24s 172ms/step - loss: 4.9019e-04 - mean_squared_error: 4.9019e-04 - val_loss: 5.2544e-04 - val_mean_squared_error: 5.2544e-04
Epoch 92/150
140/140 [==============================] - 24s 172ms/step - loss: 4.6046e-04 - mean_squared_error: 4.6046e-04 - val_loss: 5.5606e-04 - val_mean_squared_error: 5.5606e-04
Epoch 93/150
140/140 [==============================] - 23s 167ms/step - loss: 4.2890e-04 - mean_squared_error: 4.2890e-04 - val_loss: 4.4341e-04 - val_mean_squared_error: 4.4341e-04
Epoch 94/150
140/140 [==============================] - 23s 166ms/step - loss: 4.6042

140/140 [==============================] - 30s 213ms/step - loss: 5.9822e-04 - mean_squared_error: 5.9822e-04 - val_loss: 6.7202e-04 - val_mean_squared_error: 6.7202e-04
Epoch 29/150
140/140 [==============================] - 30s 215ms/step - loss: 6.2100e-04 - mean_squared_error: 6.2100e-04 - val_loss: 7.0297e-04 - val_mean_squared_error: 7.0297e-04
Epoch 30/150
140/140 [==============================] - 31s 218ms/step - loss: 5.2449e-04 - mean_squared_error: 5.2449e-04 - val_loss: 5.8084e-04 - val_mean_squared_error: 5.8084e-04
Epoch 31/150
140/140 [==============================] - 30s 217ms/step - loss: 6.0161e-04 - mean_squared_error: 6.0161e-04 - val_loss: 8.8100e-04 - val_mean_squared_error: 8.8100e-04
Epoch 32/150
140/140 [==============================] - 30s 215ms/step - loss: 5.5473e-04 - mean_squared_error: 5.5473e-04 - val_loss: 6.3251e-04 - val_mean_squared_error: 6.3251e-04
Epoch 33/150
140/140 [==============================] - 30s 217ms/step - loss: 6.0779e-04 - mean_s

### Evaluación del Modelo

#### Se carga el modelo

In [ ]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}_{I}_0.h5')
model.summary()

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_squared_error')

In [ ]:
model.save(f'../Models/{dataset_name}_{I}_{0}.h5')

In [ ]:
print(f'../Models/{dataset_name}_{I}_{0}.h5')

#### Evaluación con dataset de prueba

In [ ]:
%%time
N = 0
for i in range(len(X_test)):
    N+=len(X_test[i])
N=N/len(X_test)

n_batches = np.ceil(N/batch_size)
losses = model.evaluate(dataset_test, steps = n_batches)
K = df_desc[rpm_list[0]][7]-df_desc[rpm_list[0]][3] #Ganancia del actuador
print(f'K={"{:.2f}".format(K)}')
if not type(metrics) == list:
    metrics = [metrics]
for i, l in enumerate(['loss']+metrics):
    print(f'{l}: {"{:.2e}".format(losses[i])} -> {"{:.2f}".format(losses[i]*K)} RPM')

#### Evaluación con 1 trayectoria

In [ ]:
window_test = 3000
test_traj_generator  = DataGenerator(X=X_test,  Y=Y_test,  batch_size=1, window=window_test, sequence_out=True, feedback=feedback)

In [ ]:
%%time
X, Y = next(test_traj_generator.build_data())
x = X[0]
y = Y[0]
y_pred = []
for i in range(0, len(x)):
    if i == 0:
        x_temp = x[0].reshape(1,  1, X.shape[2])
    elif i<=window:
        x_temp = x[0:i].reshape(1,  i, X.shape[2])
    else:
        x_temp = x[i-window:i].reshape(1,  window, X.shape[2])
    y_temp = model.predict(x_temp)
    y_pred.append(y_temp)
y_pred = np.array(y_pred).reshape(y.shape)

#### Visualización con 1 trayectoria

In [ ]:
t = np.arange(0, len(y)*Ts, Ts)

In [ ]:
fig, axs = plt.subplots(Y.shape[2], figsize = (7,15))
for i in range(Y.shape[2]):
    axs[i].plot(t, y_pred[:,i], t, y[:,i])
    axs[i].grid()

In [ ]:
fig, axs = plt.subplots(Y.shape[2], figsize = (7,15))
L1 = 4*len(t)//5
L2 = 5*len(t)//5
for i in range(Y.shape[2]):
    axs[i].plot(t[L1:L2], y_pred[L1:L2,i], t[L1:L2:], y[L1:L2:,i])
    axs[i].grid()

#### Tiempo de inferencia tamaño entrada

In [ ]:
import time

def MSE(y, y_pred):
    return (y-y_pred)**2

window_test = 1000
test_traj_batch = 1000
test_traj_generator  = DataGenerator(X=X_test,  Y=Y_test,  batch_size=test_traj_batch, window=window_test, sequence_out=True)

window_len = []
inf_time = []
loss = []

x, y = next(test_traj_generator.build_data())

In [ ]:
%%time
accum_time = 0
n_iter = 200
test_range = list(set(np.rint(np.logspace(0, np.log10(x.shape[1]-1), num=n_iter, endpoint=True))))
test_range.sort()
n_iter = len(test_range)
for k, i in enumerate(list(map(int,test_range))):
    inf_time_aux = []    # Tiempo de inferencia auxiliar
    loss_aux = []    # Costo Auxiliar
    init_time = time.time()
    for j in range(len(x)):
        x_5 = x[j][0:i].reshape(1,  i, x.shape[2])
        start_time = time.time()
        y_pred = model.predict(x_5)
        finish_time = time.time() - start_time
        inf_time_aux.append(finish_time)
        loss_aux.append(
            np.mean(
                MSE(y[j][i], y_pred)
            )
        )
    
    window_len.append(i)
    inf_time.append(np.mean(inf_time_aux))
    loss.append(np.mean(loss_aux))
    clear_output(wait=True)
    accum_time += time.time()-init_time
    print(f'iter = {k} de {len(test_range)}, i = {i}, execution time = {"{:.2f}".format(np.max(accum_time))}s')

In [ ]:
inf_time_norm = np.array(inf_time)/max(inf_time)
loss_norm = np.array(loss)/max(loss)
performance = 1/(np.array(loss)*np.array(inf_time))
performance = np.array(performance)/max(performance)

In [ ]:
plt.figure()
plt.plot(window_len, inf_time_norm, label='Normalized inference time', alpha=0.7)
plt.plot(window_len, loss_norm, label='Normalized average MSE', alpha=0.7)
plt.plot(window_len, performance, label='Performance=1/(MSE*Time)', alpha=0.7)
plt.xlabel("Window Size")
plt.legend(loc=1)
plt.grid()

print(f'Best window inference time={np.argmin(inf_time_norm)} steps, time={"{:.2f}".format(np.min(inf_time)*1000)} ms')
print(f'Best window MSE loss={np.argmin(loss_norm)} steps, MSE={"{:.2e}".format(np.min(loss))}')
print(f'Best window Performance (MSExTime)={np.argmax(performance)} steps, Value={"{:.2e}".format(np.max(loss)/np.min(inf_time))}')

In [ ]:
plt.figure()
plt.semilogx(window_len, inf_time_norm, label='Normalized inference time', alpha=0.7)
plt.semilogx(window_len, loss_norm, label='Normalized average MSE', alpha=0.7)
plt.semilogx(window_len, performance, label='Performance=1/(MSE*Time)', alpha=0.7)
plt.xlabel("Window Size")
plt.legend(loc=1)
plt.grid()

In [ ]:
plt.figure()
plt.scatter(loss_norm, inf_time_norm, alpha=0.7)